# Optimización de Talento — 

- **Imputamos nulos**: numéricas → **mediana**; categóricas → **"desconocido"**.
- Mantenemos:
  - Texto en **minúsculas**, espacios → `_`, y `__` → `_`.
  - Limpieza de números con símbolos/formatos: **coma → punto** para decimal.
  - Columnas no negativas donde tenga sentido.
  - Columna **`sameasmonthlyincome`** con el mismo tratamiento que `monthlyincome`.

**Índice:**
1) Importaciones  
2) Preparar carpetas  
3) Leer CSV  
4) EDA básico  
5) Copia para limpieza  
6) Normalización de TEXTO  
7) Función de limpieza de NÚMEROS con símbolos  
8) Aplicar limpieza a NÚMEROS (incluye `sameasmonthlyincome`)  
9) Reglas adicionales (no negativos)  
10) Correcciones específicas (`gender`, `maritalstatus`)  
11) **Imputación de nulos** (mediana / "desconocido")  
12) Duplicados y guardado del CSV limpio  
13) Visualizaciones  


In [38]:
# =========================
# 1) IMPORTACIONES
# =========================
import os, re, sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ajustes de impresión
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

In [39]:
# =========================
# 2) PREPARAR CARPETAS
# =========================

# Imprimo la carpeta donde está corriendo el cuaderno para orientarnos.
print("Carpeta de trabajo:", os.getcwd())

# Defino la carpeta raíz donde guardaremos los datos del proyecto.
carpeta_data = "data"

# Dentro de 'data', creo una subcarpeta para los datos en bruto (sin tocar).
carpeta_raw = os.path.join(carpeta_data, "raw")

# Otra subcarpeta para los datos procesados/limpios (los resultados de la limpieza).
carpeta_processed = os.path.join(carpeta_data, "processed")

# Carpeta para guardar salidas varias (informes, textos, etc.).
carpeta_outputs = "outputs"

# Dentro de 'outputs', una subcarpeta específica para imágenes/gráficos.
carpeta_plots = os.path.join(carpeta_outputs, "plots")

# Carpeta donde guardaremos el fichero de la base de datos SQLite (.db).
carpeta_db = "db"

# Recorro la lista de todas las carpetas que quiero que existan
# y las creo si no existen. 'exist_ok=True' evita error si ya estaban creadas.
for p in [carpeta_raw, carpeta_processed, carpeta_outputs, carpeta_plots, carpeta_db]:
    os.makedirs(p, exist_ok=True)

# Mensaje final para confirmar las rutas que han quedado listas.
print("Carpetas listas:", [carpeta_raw, carpeta_processed, carpeta_outputs, carpeta_plots, carpeta_db])

Carpeta de trabajo: /Users/lauraparejo/Desktop/Proyecto n3/-project-da-promo-54-modulo-3-team-2/notebooks
Carpetas listas: ['data/raw', 'data/processed', 'outputs', 'outputs/plots', 'db']


In [40]:
# =========================
# 3) LEER CSV
# =========================

# Definimos primero la ruta del CSV en dos posibles ubicaciones:
# - 'ruta_local': por si el archivo está en la misma carpeta que el notebook.
# - 'ruta_raw': por si lo guardamos en la carpeta 'data/raw' del proyecto.
ruta_local = "hr_raw_data.csv"
ruta_raw = os.path.join(carpeta_raw, "hr_raw_data.csv")

# Elegimos qué ruta usar:
# - Si existe el archivo en local, usamos esa.
# - Si no, intentamos con la ruta de 'data/raw'.
ruta_csv = ruta_local if os.path.isfile(ruta_local) else ruta_raw

# Si no encontramos el archivo en ninguna de las dos rutas, lanzamos un error.
# Así evitamos que pandas intente leer un archivo inexistente.
if not os.path.isfile(ruta_csv):
    raise FileNotFoundError("No encuentro 'hr_raw_data.csv'. Ponlo junto al cuaderno o en 'data/raw/'.")

# Leemos el CSV con pandas y lo guardamos en 'df_raw'.
df_raw = pd.read_csv(ruta_csv)

# Guardamos una copia del CSV dentro de 'data/raw' para que siempre esté ordenado en el proyecto.
# Esto es útil si alguien del equipo lo tenía en otra carpeta, así todos lo tenemos en el mismo sitio.
df_raw.to_csv(ruta_raw, index=False)

# Mostramos por pantalla el tamaño del DataFrame bruto: nº de filas y columnas.
print("Tamaño DF bruto:", df_raw.shape)

# Vemos las primeras 10 filas del dataset para hacernos una idea rápida de cómo llegan los datos.
df_raw.head(10)


Tamaño DF bruto: (1678, 42)


,Unnamed: 0,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,0,51,No,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full Time,0,NaN,5,"3,0",20,NaN,15,15,"16280,83$",1972,"195370,00$",NaN,NaN,Yes
1,1,52,No,NaN,2063.388889,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,"43331,17$",0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,NaN,1971,"199990,00$",NaN,NaN,1
2,2,42,No,travel_rarely,1984.253968,Research & Development,4,2,Technical Degree,1,3,3,0,NaN,3,5,ManaGER,4,Married,NaN,"41669,33$",1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,NaN,1981,"192320,00$",ManaGER - Research & Development,NaN,1
3,3,47,No,travel_rarely,1771.404762,NaN,2,4,Medical,1,4,1,1,NaN,3,4,ReseArCH DIrECtOr,3,Married,"14307,50$","37199,50$",3,Y,NaN,19,"3,0",2,Full Time,2,NaN,2,NaN,20,NaN,5,6,"14307,50$",1976,"171690,00$",NaN,NaN,False
4,4,46,No,NaN,1582.771346,NaN,3,3,Technical Degree,1,5,1,1,NaN,4,4,sAleS EXECUtIve,1,Divorced,"12783,92$","33238,20$",2,Y,No,12,"3,0",4,NaN,1,NaN,5,"3,0",19,NaN,2,8,"12783,92$",1977,NaN,NaN,NaN,0
5,5,48,No,NaN,1771.920635,Research & Development,22,3,Medical,1,6,4,1,NaN,3,4,MANAger,4,NaN,"14311,67$","37210,33$",3,NaN,No,11,"3,0",2,NaN,1,NaN,3,"3,0",22,NaN,4,7,"14311,67$",1975,NaN,MANAger - Research & Development,NaN,Yes
6,6,59,No,NaN,1032.487286,NaN,25,3,Life Sciences,1,7,1,1,NaN,3,3,Sales ExeCutIVe,1,NaN,"8339,32$","21682,23$",7,Y,NaN,11,"3,0",4,Part Time,0,"28,0",3,"2,0",21,NaN,7,9,"8339,32$",1964,"100071,84$",NaN,NaN,True
7,7,42,No,travel_rarely,556.256661,NaN,1,1,NaN,1,8,2,0,69.532083,3,2,Sales eXEcUTiVe,3,Married,NaN,"11681,39$",1,NaN,No,25,"4,0",3,Part Time,0,"20,0",3,"3,0",20,NaN,11,6,NaN,1981,"53914,11$",NaN,NaN,0
8,8,41,No,NaN,1712.182540,NaN,2,5,NaN,1,9,2,1,NaN,3,4,mANAGEr,1,Married,"13829,17$","35955,83$",7,NaN,No,16,"3,0",2,Full Time,1,"22,0",2,"3,0",18,NaN,11,8,"13829,17$",1982,"165950,00$",NaN,NaN,True
9,9,41,No,travel_frequently,1973.984127,NaN,9,3,NaN,1,10,1,0,NaN,3,5,reSEaRCH DIrectoR,3,NaN,"15943,72$","41453,67$",2,NaN,No,17,"3,0",2,NaN,1,"21,0",2,"4,0",18,NaN,0,11,"15943,72$",1982,NaN,NaN,NaN,0


In [41]:
# =========================
# 4) EDA BÁSICO
# =========================

# 1) Mostramos por pantalla los tipos de datos de cada columna
# Esto nos sirve para ver si una columna que debería ser numérica está como texto, o al revés.
print("Tipos de columnas:")
print(df_raw.dtypes)

# 2) Mostramos el número de valores nulos por cada columna
# Esto nos ayuda a detectar qué columnas necesitan imputación o limpieza.
print("\nNulos por columna:")
print(df_raw.isna().sum())

# 3) Contamos cuántas filas duplicadas exactas hay en el DataFrame
# Esto es importante para evitar contar empleados más de una vez.
print("\nDuplicados exactos:", df_raw.duplicated().sum())

# 4) Guardamos toda esta misma información en un archivo de texto dentro de la carpeta 'outputs'.
# Así tenemos un registro escrito que podemos revisar sin necesidad de ejecutar el notebook.
with open(os.path.join(carpeta_outputs, "eda_basico_v5.txt"), "w", encoding="utf-8") as f:
    # Guardamos los tipos de columnas
    f.write("TIPOS:\n")
    f.write(str(df_raw.dtypes))
    f.write("\n\n")

    # Guardamos los nulos por columna
    f.write("NULOS:\n")
    f.write(str(df_raw.isna().sum()))
    f.write("\n")

    # Guardamos el número de duplicados
    f.write("DUPLICADOS:\n")
    f.write(str(df_raw.duplicated().sum()))


Tipos de columnas:
Unnamed: 0                    int64
age                          object
attrition                    object
businesstravel               object
dailyrate                   float64
department                   object
distancefromhome              int64
education                     int64
educationfield               object
employeecount                 int64
employeenumber                int64
environmentsatisfaction       int64
gender                        int64
hourlyrate                  float64
jobinvolvement                int64
joblevel                      int64
jobrole                      object
jobsatisfaction               int64
maritalstatus                object
monthlyincome                object
monthlyrate                  object
numcompaniesworked            int64
over18                       object
overtime                     object
percentsalaryhike             int64
performancerating            object
relationshipsatisfaction      int64
standardh

In [42]:
# =========================
# 4) EDA INFO
# =========================

# La función .info() nos da un resumen muy útil del DataFrame:
# - El número total de filas y columnas
# - El nombre de cada columna
# - Cuántos valores NO nulos tiene cada columna
# - El tipo de dato de cada columna (int, float, object, etc.)
# - Y al final, el consumo aproximado de memoria

df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1678 entries, 0 to 1677
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1678 non-null   int64  
 1   age                       1678 non-null   object 
 2   attrition                 1678 non-null   object 
 3   businesstravel            877 non-null    object 
 4   dailyrate                 1678 non-null   float64
 5   department                312 non-null    object 
 6   distancefromhome          1678 non-null   int64  
 7   education                 1678 non-null   int64  
 8   educationfield            904 non-null    object 
 9   employeecount             1678 non-null   int64  
 10  employeenumber            1678 non-null   int64  
 11  environmentsatisfaction   1678 non-null   int64  
 12  gender                    1678 non-null   int64  
 13  hourlyrate                411 non-null    float64
 14  jobinvol

In [43]:
# =========================
# 5) COPIA PARA LIMPIEZA
# =========================

# 1) Creamos una copia del DataFrame original 'df_raw'.
# Esto lo hacemos para no modificar los datos originales mientras limpiamos.
# Así, si nos equivocamos o necesitamos comparar, siempre tenemos el dataset original intacto.
df = df_raw.copy()

# 2) Mostramos el tamaño (número de filas y columnas) del DataFrame de trabajo.
# df.shape devuelve una tupla (n_filas, n_columnas).
print("Tamaño inicio limpieza:", df.shape)

# 3) Mostramos las primeras 5 filas del DataFrame.
# Esto nos sirve para echar un vistazo rápido al estado inicial antes de aplicar transformaciones.
df.head(5)


Tamaño inicio limpieza: (1678, 42)


,Unnamed: 0,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,0,51,No,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full Time,0,NaN,5,"3,0",20,NaN,15,15,"16280,83$",1972,"195370,00$",NaN,NaN,Yes
1,1,52,No,NaN,2063.388889,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,"43331,17$",0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,NaN,1971,"199990,00$",NaN,NaN,1
2,2,42,No,travel_rarely,1984.253968,Research & Development,4,2,Technical Degree,1,3,3,0,NaN,3,5,ManaGER,4,Married,NaN,"41669,33$",1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,NaN,1981,"192320,00$",ManaGER - Research & Development,NaN,1
3,3,47,No,travel_rarely,1771.404762,NaN,2,4,Medical,1,4,1,1,NaN,3,4,ReseArCH DIrECtOr,3,Married,"14307,50$","37199,50$",3,Y,NaN,19,"3,0",2,Full Time,2,NaN,2,NaN,20,NaN,5,6,"14307,50$",1976,"171690,00$",NaN,NaN,False
4,4,46,No,NaN,1582.771346,NaN,3,3,Technical Degree,1,5,1,1,NaN,4,4,sAleS EXECUtIve,1,Divorced,"12783,92$","33238,20$",2,Y,No,12,"3,0",4,NaN,1,NaN,5,"3,0",19,NaN,2,8,"12783,92$",1977,NaN,NaN,NaN,0


In [44]:
# =========================
# 12) LIMPIEZA: ELIMINAR COLUMNAS
# =========================

# Lista de columnas que queremos eliminar
columnas_a_eliminar = [
    "employeecount",
    "yearsincurrentrole",
    "sameasmonthlyincome",
    "roledepartament",
    "numberchildren"
]

# Comprobamos cuáles de esas columnas están realmente en el DataFrame
columnas_presentes = [col for col in columnas_a_eliminar if col in df.columns]

# Si hay columnas presentes, las eliminamos
if columnas_presentes:
    df.drop(columns=columnas_presentes, inplace=True)
    print(f"Columnas eliminadas: {columnas_presentes}")
else:
    print("Ninguna de las columnas especificadas está presente en el DataFrame.")

Columnas eliminadas: ['employeecount', 'yearsincurrentrole', 'sameasmonthlyincome', 'roledepartament', 'numberchildren']


In [45]:
# =========================
# 6) NORMALIZACIÓN DE TEXTO
# =========================
# Regla general para TODAS las columnas de texto:
# - a string "seguro"
# - strip() a los lados
# - lower() a minúsculas
# - espacio -> '_' (sin regex para que sea literal)
# - '__' -> '_' (colapsar)
# - quitar '_' al principio y al final

for col in df.columns: # para todas las columnas que recorras del DF 
    if pd.api.types.is_string_dtype(df[col]): # solo entra a limpiar si la columna es de tipo texto (string, object).
        df[col] = df[col].astype("string")
        df[col] = df[col].str.strip()
        df[col] = df[col].str.lower()
        df[col] = df[col].str.replace(" ", "_", regex=False)
        df[col] = df[col].str.replace(r"_+", "_", regex=True)
        df[col] = df[col].str.strip("_")
         # EXTRA: convertir vacíos y literales comunes a NaN para que la imputación funcione
        df[col] = df[col].replace({"": pd.NA, "nan": pd.NA, "none": pd.NA})

print("Normalización de texto aplicada.")

Normalización de texto aplicada.


In [46]:
# =========================
# 7) FUNCIÓN: limpiar números con símbolos
# =========================

def limpiar_numero_texto(x):
    # Primero compruebo si el valor es nulo (NaN o pd.NA)
    # Si lo es, lo devuelvo tal cual porque ya lo imputaremos más tarde
    if pd.isna(x):
        return x

    # Convierto el valor a string para poder manipularlo con replace()
    s = str(x)

    # Quito espacios, símbolos de moneda ($, €) y porcentajes (%)
    s = s.replace(" ", "").replace("$", "").replace("€", "").replace("%","")

    # Si hay coma y punto a la vez, tengo que decidir cuál es el decimal
    if "," in s and "." in s:
        import re
        # Caso europeo: si termina en coma + 1 o 2 dígitos (ej. "1.234,56")
        if re.search(r",\d{1,2}$", s):
            # Quito los puntos de miles y convierto la coma en punto decimal
            s = s.replace(".", "").replace(",", ".")
        else:
            # Caso americano: la coma es separador de miles, la borro
            # Ejemplo: "1,234.56" -> "1234.56"
            s = s.replace(",", "")
    else:
        # Si solo hay comas y ningún punto
        if "," in s and "." not in s:
            # Asumo que la coma es decimal -> la cambio a punto
            # También elimino puntos sueltos por si acaso
            s = s.replace(".", "").replace(",", ".")

    # Limpieza final con regex:
    # Borro cualquier cosa que no sea número o punto decimal
    import re
    s = re.sub(r"[^0-9.]", "", s)

    # Devuelvo el string limpio (más tarde lo convertiremos a float)
    return s

print("Función 'limpiar_numero_texto' lista.")


Función 'limpiar_numero_texto' lista.


In [47]:
# =========================
# 8) LIMPIEZA NÚMEROS (símbolos/comas/puntos)
# =========================

# Primero hago una lista con los nombres de las columnas que deberían ser numéricas en nuestro dataset.
# Estas columnas, en bruto, pueden venir con símbolos de dinero, comas en lugar de puntos, etc.
# También incluyo 'sameasmonthlyincome', que debe tratarse igual que 'monthlyincome'.
columnas_numericas_esperadas = [
    "dailyrate","hourlyrate","monthlyincome","sameasmonthlyincome","monthlyrate",
    "percentsalaryhike","stockoptionlevel","distancefromhome",
    "age","education","joblevel","totalworkingyears",
    "yearsatcompany","yearsincurrentrole","yearssincelastpromotion",
    "yearswithcurrmanager","numberchildren","salary","trainingtimeslastyear",
    "environmentsatisfaction","jobinvolvement","jobsatisfaction",
    "relationshipsatisfaction","performancerating","worklifebalance"
]

# Como en el dataset los nombres de columnas pueden variar en mayúsculas/minúsculas,
# creo un diccionario que relaciona el nombre en minúsculas con el nombre real en el DataFrame.
# Ejemplo: si la columna se llama "DailyRate", este mapa me devuelve "DailyRate" cuando busco "dailyrate".
lower_map = {c.lower(): c for c in df.columns}

# Ahora recorro cada nombre de la lista que hice arriba.
for cname in columnas_numericas_esperadas:
    # Si esa columna existe realmente en el DataFrame (aunque esté en mayúsculas),
    # entonces la limpiamos.
    if cname in lower_map:
        # Recupero el nombre real de la columna
        col = lower_map[cname]

        # 1) Convierto la columna a texto (string) para poder aplicar la función de limpieza sin errores.
        df[col] = df[col].astype("string")

        # 2) Aplico nuestra función 'limpiar_numero_texto' que elimina símbolos ($, €, %) y arregla comas/puntos.
        df[col] = df[col].apply(limpiar_numero_texto)

        # 3) Convierto el resultado a número real (float).
        #    Si alguna celda no se puede convertir, se pone como NaN automáticamente.
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Mensaje final para confirmar que la limpieza se ha aplicado a todas las columnas numéricas.
print("Limpieza numérica aplicada (decimal='.').")


Limpieza numérica aplicada (decimal='.').


In [48]:
# =========================
# 9) REGLA: no negativos
# =========================

# Hay columnas numéricas que, por lógica, nunca deberían tener valores negativos.
# Ejemplos:
# - DistanceFromHome (distancia a casa) → no puede ser negativa.
# - DailyRate, HourlyRate, MonthlyIncome, SameAsMonthlyIncome, MonthlyRate, Salary → son sueldos/tarifas → siempre ≥ 0.

for cname in ["distancefromhome","dailyrate","hourlyrate","monthlyincome","sameasmonthlyincome","monthlyrate","salary"]:
    # Primero compruebo si esa columna existe en el DataFrame (por eso miro en el diccionario lower_map).
    if cname in lower_map:
        # Recupero el nombre real de la columna (con mayúsculas/minúsculas como esté en el CSV).
        col = lower_map[cname]

        # Compruebo si la columna es numérica de verdad.
        if pd.api.types.is_numeric_dtype(df[col]):
            # Aplico el valor absoluto a toda la columna.
            # Esto convierte números negativos en positivos (ej: -5 → 5).
            df[col] = df[col].abs()

# Mensaje final de confirmación para saber que la regla ya se aplicó.
print("Columnas no-negativas aseguradas.")


Columnas no-negativas aseguradas.


In [49]:
# =========================
# 10) CORRECCIONES ESPECÍFICAS
# =========================

# --- Corrección 1: columna 'gender'
# En algunos datasets, 'gender' aparece como 0/1 en vez de 'male'/'female'.
# Por claridad, convertimos 1 -> "male" y 0 -> "female".
if "gender" in lower_map:                       # solo si existe la columna gender
    col = lower_map["gender"]                   # recupero el nombre real de la columna
    if pd.api.types.is_numeric_dtype(df[col]):  # si es numérica (0/1)
        df[col] = df[col].map({1:"male", 0:"female"})
    else:
        # Si viene como texto "0" o "1" (string), también lo convertimos
        df[col] = df[col].replace({"1":"male","0":"female",1:"male",0:"female"})

# --- Corrección 2: columna 'maritalstatus'
# A veces hay un error tipográfico: en vez de "married" aparece "marreid".
# Lo corregimos para que todos los valores estén uniformes.
if "maritalstatus" in lower_map:                        # solo si existe la columna maritalstatus
    col = lower_map["maritalstatus"]
    # Convertimos a string y reemplazamos "marreid" por "married"
    df[col] = df[col].astype("string").str.replace("marreid","married",regex=False)

# Mensaje de confirmación para saber que ya hicimos estas correcciones puntuales.
print("Correcciones (gender, maritalstatus) aplicadas.")


Correcciones (gender, maritalstatus) aplicadas.


In [50]:
# =========================
# 11) IMPUTACIÓN DE NULOS (robusta)
# =========================
import numpy as np
import pandas as pd

# 1) Detectamos numéricas y categóricas por dtype
num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = df.select_dtypes(exclude=[np.number]).columns

# 2) Lista de columnas que deben ser NO NEGATIVAS (para elegir fallback sensato si todo es NaN)
no_negativas = {
    "distancefromhome","dailyrate","hourlyrate","monthlyincome","sameasmonthlyincome",
    "monthlyrate","salary","percentsalaryhike","stockoptionlevel","numberchildren",
    "trainingtimeslastyear","yearsatcompany","yearsincurrentrole","yearssincelastpromotion",
    "yearswithcurrmanager","totalworkingyears","age","education","joblevel",
}

# 3) Imputación
for c in num_cols:
    if df[c].isna().all():
        # Si toda la columna numérica es NaN, usamos fallback:
        fill_value = 0.0 if c.lower() in no_negativas else 0.0  # cambia aquí si prefieres otro valor
    else:
        fill_value = df[c].median(skipna=True)
    df[c] = df[c].fillna(fill_value)

for c in cat_cols:
    df[c] = df[c].fillna("desconocido")

print("Imputación completada (numéricas=mediana/fallback, categóricas='desconocido').")


Imputación completada (numéricas=mediana/fallback, categóricas='desconocido').


In [51]:
# =========================
# 12) DUPLICADOS Y GUARDADO
# =========================

# 1) Eliminamos filas duplicadas en el DataFrame.
# 'drop_duplicates' detecta filas exactamente iguales en todas las columnas.
# Con 'keep="first"' nos quedamos con la primera aparición y borramos las siguientes.
df = df.drop_duplicates(keep="first")

# 2) Definimos la ruta donde queremos guardar el CSV limpio.
# Usamos la carpeta 'processed' que creamos al inicio del proyecto para separar datos limpios.
ruta_csv_limpio = os.path.join(carpeta_processed, "hr_clean_data_v5.csv")

# 3) Guardamos el DataFrame limpio en un CSV nuevo.
# 'index=False' es para que no añada la columna extra del índice de pandas en el archivo.
df.to_csv(ruta_csv_limpio, index=False)

# 4) Imprimimos un mensaje para confirmar que el archivo se ha guardado correctamente.
print("CSV limpio v5 guardado en:", ruta_csv_limpio)

# 5) Mostramos las primeras 10 filas del DataFrame limpio para revisar cómo quedó.
df.head(10)


CSV limpio v5 guardado en: data/processed/hr_clean_data_v5.csv


,Unnamed: 0,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework
0,0,51.0,no,desconocido,2015.722222,desconocido,6,3,desconocido,1,1,female,69.532083,3,5,research_director,3,desconocido,16280.83,42330.17,7,Y,No,13,3.0,3,Full Time,0,10.0,5,3.0,20,15,15,1972,195370.00,yes
1,1,52.0,no,desconocido,2063.388889,desconocido,1,4,Life Sciences,2,3,female,69.532083,2,5,manager,3,desconocido,4492.84,43331.17,0,desconocido,desconocido,14,3.0,1,desconocido,1,34.0,5,3.0,33,11,9,1971,199990.00,1
2,2,42.0,no,travel_rarely,1984.253968,Research & Development,4,2,Technical Degree,3,3,female,69.532083,3,5,manager,4,Married,4492.84,41669.33,1,desconocido,No,11,3.0,4,desconocido,0,22.0,3,3.0,22,11,15,1981,192320.00,1
3,3,47.0,no,travel_rarely,1771.404762,desconocido,2,4,Medical,4,1,male,69.532083,3,4,research_director,3,Married,14307.50,37199.50,3,Y,desconocido,19,3.0,2,Full Time,2,10.0,2,3.0,20,5,6,1976,171690.00,false
4,4,46.0,no,desconocido,1582.771346,desconocido,3,3,Technical Degree,5,1,male,69.532083,4,4,sales_executive,1,Divorced,12783.92,33238.20,2,Y,No,12,3.0,4,desconocido,1,10.0,5,3.0,19,2,8,1977,53914.11,0
5,5,48.0,no,desconocido,1771.920635,Research & Development,22,3,Medical,6,4,male,69.532083,3,4,manager,4,desconocido,14311.67,37210.33,3,desconocido,No,11,3.0,2,desconocido,1,10.0,3,3.0,22,4,7,1975,53914.11,yes
6,6,59.0,no,desconocido,1032.487286,desconocido,25,3,Life Sciences,7,1,male,69.532083,3,3,sales_executive,1,desconocido,8339.32,21682.23,7,Y,desconocido,11,3.0,4,Part Time,0,28.0,3,2.0,21,7,9,1964,100071.84,true
7,7,42.0,no,travel_rarely,556.256661,desconocido,1,1,desconocido,8,2,female,69.532083,3,2,sales_executive,3,Married,4492.84,11681.39,1,desconocido,No,25,4.0,3,Part Time,0,20.0,3,3.0,20,11,6,1981,53914.11,0
8,8,41.0,no,desconocido,1712.182540,desconocido,2,5,desconocido,9,2,male,69.532083,3,4,manager,1,Married,13829.17,35955.83,7,desconocido,No,16,3.0,2,Full Time,1,22.0,2,3.0,18,11,8,1982,165950.00,true
9,9,41.0,no,travel_frequently,1973.984127,desconocido,9,3,desconocido,10,1,female,69.532083,3,5,research_director,3,desconocido,15943.72,41453.67,2,desconocido,No,17,3.0,2,desconocido,1,21.0,2,4.0,18,0,11,1982,53914.11,0


In [52]:
# =========================
# 13) VISUALIZACIONES
# =========================

# --- Visualización 1: Gráfico de barras de 'attrition' ---
# Comprobamos si existe la columna 'attrition' en el DataFrame
if "attrition" in df.columns:
    plt.figure()  # abrimos una figura nueva
    # Contamos los valores de 'attrition' (sí/no) y los mostramos en barras
    df["attrition"].value_counts(dropna=False).plot(kind="bar")
    plt.title("Attrition (conteo)")             # título del gráfico
    plt.xlabel("attrition")                     # etiqueta eje X
    plt.ylabel("n")                             # etiqueta eje Y
    # Guardamos la imagen en la carpeta 'plots' para tenerla disponible
    p1 = os.path.join(carpeta_plots, "attrition_counts_v5.png")
    plt.savefig(p1, bbox_inches="tight")
    plt.close()
    print("Gráfico:", p1)                       # confirmamos la ruta

# --- Visualización 2: Histograma de 'monthlyincome' ---
# Comprobamos si existe la columna 'monthlyincome'
if "monthlyincome" in df.columns:
    plt.figure()  # abrimos figura nueva
    # Dibujamos un histograma con 30 bins (intervalos)
    df["monthlyincome"].astype(float).plot(kind="hist", bins=30)
    plt.title("MonthlyIncome (histograma)")
    plt.xlabel("monthlyincome")                 # eje X: ingresos
    plt.ylabel("frecuencia")                    # eje Y: frecuencia
    # Guardamos la imagen
    p2 = os.path.join(carpeta_plots, "monthlyincome_hist_v5.png")
    plt.savefig(p2, bbox_inches="tight")
    plt.close()
    print("Gráfico:", p2)

# --- Visualización 3: Boxplot de 'yearsatcompany' por 'department' ---
# Solo si tenemos ambas columnas disponibles
if "yearsatcompany" in df.columns and "department" in df.columns:
    plt.figure()
    data = []    # lista con valores de cada departamento
    labels = []  # nombres de los departamentos

    # Agrupamos por departamento y recogemos los valores de 'yearsatcompany'
    for d, sub in df.groupby("department"):
        vals = sub["yearsatcompany"].astype(float).values
        if len(vals) > 0:
            data.append(vals)
            labels.append(str(d))

    # Si tenemos datos, dibujamos el boxplot comparativo
    if data:
        plt.boxplot(data, labels=labels, vert=True)
        plt.xticks(rotation=45, ha="right")     # giramos etiquetas del eje X
        plt.tight_layout()                      # ajustamos para que no se solapen
        # Guardamos la imagen
        p3 = os.path.join(carpeta_plots, "yearsatcompany_by_department_v5.png")
        plt.savefig(p3, bbox_inches="tight")
        plt.close()
        print("Gráfico:", p3)

Gráfico: outputs/plots/attrition_counts_v5.png
Gráfico: outputs/plots/monthlyincome_hist_v5.png
Gráfico: outputs/plots/yearsatcompany_by_department_v5.png


/var/folders/7z/nkwh7lln1mx_fzrt6mjwyxzr0000gp/T/ipykernel_29653/496719430.py:51: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=labels, vert=True)
